In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')
import numpy as np
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
file = open('data.pkl', 'rb')
data = pickle.load(file)

In [3]:
data

,body_basic,label
0,Fly fishing is an angling method that uses a ...,fly_fishing
1,Simms GORE TEX ExStream Cap 89 95 Simms Dockwe...,fly_fishing
2,Article NPS Photo Neal Herbert The thick yello...,fly_fishing
3,Fly Fishing is a technique for catching fish w...,fly_fishing
4,11 Tips to Help You Sell a Boat Online Fishin...,fly_fishing
...,...,...
232,We use cookies to give you a better experience...,machine_learning
233,This post is part one in a three part series o...,machine_learning
234,A leading edge research firm focused on digita...,machine_learning
235,Explore Northeastern s first international cam...,machine_learning


In [4]:
nltk.download('stopwords')
stemmer = PorterStemmer()
words = stopwords.words("english")
data['cleaned'] = data['body_basic'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data

,body_basic,label,cleaned
0,Fly fishing is an angling method that uses a ...,fly_fishing,fli fish angl method use light weight lure cal...
1,Simms GORE TEX ExStream Cap 89 95 Simms Dockwe...,fly_fishing,simm gore tex exstream cap simm dockwear wool ...
2,Article NPS Photo Neal Herbert The thick yello...,fly_fishing,articl np photo neal herbert the thick yellow ...
3,Fly Fishing is a technique for catching fish w...,fly_fishing,fli fish techniqu catch fish bait often look l...
4,11 Tips to Help You Sell a Boat Online Fishin...,fly_fishing,tip help you sell boat onlin fish tackl storag...
...,...,...,...
232,We use cookies to give you a better experience...,machine_learning,we use cooki give better experi carri brows ha...
233,This post is part one in a three part series o...,machine_learning,thi post part one three part seri challeng imp...
234,A leading edge research firm focused on digita...,machine_learning,a lead edg research firm focus digit transform...
235,Explore Northeastern s first international cam...,machine_learning,explor northeastern first intern campu canada ...


In [6]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(data['cleaned']).toarray()
final_features.shape

(237, 5105)

In [7]:
vectorizer

TfidfVectorizer(min_df=3, ngram_range=(1, 2), stop_words='english',
                sublinear_tf=True)

In [8]:
from sklearn.linear_model import LogisticRegression
X = data['cleaned']
Y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', LogisticRegression(random_state=0))])

model = pipeline.fit(X_train, y_train)
with open('LogisticRegression.pkl', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

print(classification_report(ytest, model.predict(X_test)))


                  precision    recall  f1-score   support

     fly_fishing       0.88      1.00      0.94        22
      ice_hockey       1.00      0.83      0.91        18
machine_learning       1.00      1.00      1.00        20

        accuracy                           0.95        60
       macro avg       0.96      0.94      0.95        60
    weighted avg       0.96      0.95      0.95        60

